# 模拟Uni PositionValue的变化

In [1]:

import pandas as pd
import numpy as np
import math
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.io as pio
pio.renderers.default = 'notebook'

In [2]:
# 模拟币本位合约盈亏

# 生成价格序列
def gen_price_series(start_price, end_price, n):
    price_series = [start_price]
    interval = (end_price - start_price) / n
    for i in range(n):
        price_series.append(price_series[i] + interval)
    return price_series

# 计算盈亏
def calc_bi_pnl(open, close,direction,contract_value):
    return (1/open - 1/close) * direction * contract_value
def calc_u_pnl(open, close,direction,contract_value):
    return (close - open)/open * direction * contract_value
    

In [8]:
import math
p2 = 1867
p_remove = 2050
sigma = 0.05
p0 = p2 * (1-sigma)
p1 = p2 * (1+sigma)

q96 = 1 
a0 = 50
a1 = 50000

sqrtp0 = (math.sqrt(p0)* q96)
sqrtp1 = (math.sqrt(p1)* q96)
sqrtp2 = (math.sqrt(p2)* q96)

liq0 = a0 * (sqrtp2 * sqrtp1) / q96 / (sqrtp1 - sqrtp2)
liq1 = a1 * q96 / (sqrtp2 - sqrtp0)
liq = min(liq0, liq1)
print("liq is: ", liq)

rx = liq * q96 * (sqrtp1 - sqrtp2) / (sqrtp2 * sqrtp1)
ry = liq * (sqrtp2 - sqrtp0) / q96
print("rx is: ", rx)
print("ry is :", ry)

liq is:  45700.867523793364
rx is:  25.48989304821708
ry is : 50000.0


In [4]:
print(rx + (1/(p_remove**0.5) - 1/(2000**0.5)) * liq)
print('dx : ', (1/(p_remove**0.5) - 1/(2000**0.5)) * liq)
print(ry + ((p_remove**0.5) - (2000**0.5)) * liq)
print('dy : ', ((p_remove**0.5) - (2000**0.5)) * liq)
a0 = liq * p2 ** 0.5 / 2
print('bi pnl : ', calc_bi_pnl(2000, p_remove, 1, a0))
print('u pnl : ', calc_u_pnl(2000, p_remove, -1, a0))
print('sum x : ',calc_bi_pnl(2000, p_remove, 1, a0) + (1/(p_remove**0.5) - 1/(2000**0.5)) * liq)
print('sum y',calc_u_pnl(2000, p_remove, -1, a0)  +((p_remove**0.5) - (2000**0.5)) * liq)

11.679758586090017
dx :  -12.115056574420546
74531.11988446055
dy :  24531.119884460553
bi pnl :  12.04072825902987
u pnl :  -24683.492931011213
sum x :  -0.07432831539067664
sum y -152.37304655065964


In [45]:
p_remove = 1950
calc_u_pnl(2000, p_remove, -1, a0)/calc_bi_pnl(2000, p_remove, 1, a0)

-1950.0000000000018

In [46]:
(calc_u_pnl(2000, p_remove, -1, a0)  +((p_remove**0.5) - (2000**0.5)) * liq) +p_remove * (calc_bi_pnl(2000, p_remove, 1, a0) + (1/(p_remove**0.5) - 1/(2000**0.5)) * liq)

-312.46179460486957

In [47]:
# when price goes up :
average_buy_price = (p1 ** 0.5 - p2 ** 0.5) / (1/p2 ** 0.5 - 1/p1 ** 0.5)
print("average_buy_price is :", average_buy_price, "upper bound price is :", p1)

# when price goes down :
average_sell_price = (p2 ** 0.5 - p0 ** 0.5) / (1/p0 ** 0.5 - 1/p2 ** 0.5)
print("average_sell_price is :", average_sell_price, "lower bound price is :", p0)

average_buy_price is : 2049.390153191909 upper bound price is : 2100.0
average_sell_price is : 1949.3588689617975 lower bound price is : 1900.0


In [48]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# 定义函数f(p1, a)
def dx(p1,liq):
    # 这里定义函数f的具体表达式
    p0 = p2
    return (1/p1**0.5 - 1/p0**0.5)*liq
# 定义方差函数

def dy(p1,a):
    p0 = p2
    return (p1 ** 0.5 - p0 ** 0.5) * liq 

def get_p1_values(sigma):
    p0 = p2 * (1-sigma)
    p1 = p2 * (1+sigma)
    return np.linspace(p0, p1, 1000)


In [49]:
# 计算dx
eth_amount = dx(get_p1_values(sigma),liq) + rx
usdt_amount = dy(get_p1_values(sigma),liq) + ry

lp_value = eth_amount * get_p1_values(sigma) + usdt_amount
lp_value = np.array(lp_value) - rx * p2 - ry
hold_value = rx * get_p1_values(sigma) + ry 
hold_value = np.array(hold_value) - rx * p2 - ry
il = lp_value - hold_value

In [50]:
# 计算期货损益
def calc_futures_pnl(open, close, direction, contract_value):
    return (close - open) / open * direction * contract_value


In [51]:
# 生成损益序列
future_pnl1 = [calc_futures_pnl(p2, p1, -1, rx * p2) for p1 in get_p1_values(sigma)]
future_pnl2 = [calc_futures_pnl(p2, p1, 1, rx * p2) for p1 in get_p1_values(sigma)]
future_pnl = np.array(future_pnl1) + np.array(future_pnl2)

In [59]:
total_value = lp_value  + future_pnl1

In [60]:
testline = np.array(dx(get_p1_values(sigma),liq)) * np.array(get_p1_values(sigma)) + np.array(dy(get_p1_values(sigma),liq))

In [62]:
# plot total value
fig = go.Figure()
fig.add_trace(go.Scatter(x=get_p1_values(sigma), y=lp_value,
                    mode='lines',
                    name='lp value'))
fig.add_trace(go.Scatter(x=get_p1_values(sigma), y=hold_value,
                    mode= 'lines',
                    line=dict(dash='dash'),
                    name='hold value'))
fig.add_trace(go.Scatter(x=get_p1_values(sigma), y=future_pnl2,
                    mode='lines',
                    name='future pnl'))
fig.add_trace(go.Scatter(x=get_p1_values(sigma), y=il,
                    mode='lines',
                    name='impermanent loss'))

fig.add_trace(go.Scatter(x=get_p1_values(sigma), y=total_value,
                    mode='lines',
                    name='total value'))
fig.add_trace(go.Scatter(x=get_p1_values(sigma), y=il + future_pnl2,
                    mode='lines',
                    name='il + future pnl2'))

fig.update_layout(
    title="simulation of v3 and constant",
    xaxis_title="price",
    yaxis_title="value in usdt",
    legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)
fig.show()

In [56]:
c_dx = np.array(calc_bi_pnl(p2, get_p1_values(sigma), 1, liq * p2 ** 0.5 /2))
c_dy = np.array(calc_u_pnl(p2, get_p1_values(sigma), -1, liq * p2 ** 0.5 /2))

In [57]:
uni_dx = np.array(dx(get_p1_values(sigma),liq))
uni_dy = np.array(dy(get_p1_values(sigma),liq))

In [58]:
# plot 
data = [go.Scatter(x=get_p1_values(sigma), y=c_dx, name="c_dx"),
        go.Scatter(x=get_p1_values(sigma), y=c_dy, name="c_dy", yaxis="y2"),
        go.Scatter(x=get_p1_values(sigma), y=uni_dx, name="uni_dx"),
        go.Scatter(x=get_p1_values(sigma), y=uni_dy, name="uni_dy", yaxis="y2"),
        go.Scatter(x=get_p1_values(sigma), y=c_dx + uni_dx, name="dx"),
        go.Scatter(x=get_p1_values(sigma), y=c_dy + uni_dy, name="dy", yaxis="y2")]


layout = go.Layout(
    yaxis = dict(title = "c_dx, c_dy"),
    yaxis2 = dict(title = "uni_dx, uni_dy", overlaying = "y", side = "right"),
)

fig = go.Figure(data=data, layout=layout)
fig.show()
